In [14]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [15]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [16]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [17]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [18]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2025-06-05,12.279029,883.389453,815.18
1,3,2025-05-24,NaN,NaN,52.72
2,3,2025-05-26,1.065976,65.266759,14.99
3,3,2025-06-02,NaN,NaN,28.46
4,3,2025-06-05,NaN,NaN,27.13
...,...,...,...,...,...
410,609,2025-05-26,2.818424,NaN,81.98
411,609,2025-05-29,1.466327,92.234535,NaN
412,609,2025-06-02,NaN,NaN,135.84
413,609,2025-06-05,NaN,NaN,106.86


In [19]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [20]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [21]:
list_df = []

In [22]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

In [23]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [24]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2025-06-05,12.279029,883.389453,815.18
1,3,2025-05-26,1.065976,65.266759,67.71
2,3,2025-06-05,0.000000,0.000000,55.59
3,4,2025-06-05,0.000000,0.000000,343.13
4,5,2025-05-24,6.401479,376.148855,374.85
...,...,...,...,...,...
310,456,2025-06-05,0.000000,0.000000,43.82
311,457,2025-06-05,4.255772,188.985713,194.99
312,468,2025-06-02,7.053169,498.875476,721.63
313,468,2025-06-05,0.000000,0.000000,700.27


In [26]:
df_combinado.to_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')